In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

##### [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process

This process recovers the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a LED illuminator during the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
import fwdimaging as fwd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import os
import math
from PIL import Image

In [ ]:
%matplotlib inline

In [ ]:
def select_centered_subset(n_leds, leds2use, **kwargs):
    """Selects a centered subset of leds2use (int) length from a range
    between 0 and n_leds (int) or 1 and n_leds + 1.
    
    We recommend to USE IT ONLY for squared LED illuminator
    simulations."""
    firstis1 = kwargs.pop('firstis1', False)
    # Begin the selection down (int) rows down.
    down = kwargs.pop('down', 0)
    # Begin the selection right (int) columns right.
    right = kwargs.pop('right', 0)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(select_centered_subset.__name__,
                  list(kwargs.keys())[-1]))
    leds_prc = int(math.sqrt(n_leds))  # Number of LEDs per row or
                                       # column in a squared LED grid.
    new_leds_prc = int(math.sqrt(leds2use))
    if firstis1:
        # Init the sequence of integer from 1.
        arr = np.arange(1, n_leds + 1) 
    else:
        arr = np.arange(n_leds)
    # Convert arr ('numpy.ndarray') (one-dimensional) in 
    # leds_prc-by-leds_prc array (two-dimensional).
    arr = np.split(arr, leds_prc)  # Here, arr is type list.
    arr = np.array(arr)  # arr is a numpy.ndarray.
    if (leds_prc % 2 == 0 and new_leds_prc % 2 != 0 or
        leds_prc % 2 != 0 and new_leds_prc % 2 == 0):
        new_leds_prc -= 1  # leds_prc (int) and new_leds_prc (int) are
                           # even or odd.
    limit_l_up = (leds_prc - new_leds_prc)  # It is always even.
    limit_l_up //= 2
    limit_r_down = leds_prc - limit_l_up
    arr = arr[limit_l_up - down:limit_r_down - down, limit_l_up + right:limit_r_down + right]
    arr = arr.flat
    return list(arr)

In [ ]:
def get_set_from_folder(subsel, **kwargs):
    """Retrieves images from a directory and returns them as a
    three-dimentional arrays (numpy.ndarray).
    
    If this function does not find an image with the index number
    indicated in subsel (list), it add an array[(0, 0)] to the array
    of images.
    
    Args:
        subsel: A list with the indexes of the images to retrieve. An
            index must be part of the filename.
        **kwargs: Keyword arguments. See inline comments.
    Returns:
        arr_imgs = A numpy.ndarray that contains the imported images
            as non-zero arrays and array[(0, 0)] replacing to missing
            images.
        l_idx_zeros = A list with the indexes of the mising images.
    """
    # s3out (str) is the path of the outputs such as images generated
    # by SuperScanner Software (S3).
    s3out = kwargs.pop('s3out', os.path.join(os.path.expanduser('~'),
                                             's3-out'))
    dirname = kwargs.pop('dirname', 'lores-set-0001')
    # path (str) is the route of the directory that contains a
    # low-resolution image set.
    path = kwargs.pop('path', os.path.join(s3out, 'microscope',
                                           dirname))
    prefix = kwargs.pop('prefix', 'lores-img_')  # Prefix in the names
                                                 # of the input files.
    extension = kwargs.pop('extension', '.tiff')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_set_from_folder.__name__,
                  list(kwargs.keys())[-1]))
    len_suffix = len(str(max(subsel)))  # Length of the largest number
                                        # represented as a string.
    l_imgs = list()  # List to contain the imported images.
    l_idx_zeros = list()
    for i in range(len(subsel)):
        filepath = os.path.join(path,
                                prefix + fwd.num_str_zeros(subsel[i],
                                    len_suffix) + extension)
        try:
            img = Image.open(filepath)
            arr_img = np.array(img)
            print('Opened Lo-Res image:', filepath)
            img.close()
            l_imgs.append(arr_img)
        except IOError:
            print('No such file:', filepath)
            l_imgs.append(np.zeros((0, 0)))
            l_idx_zeros.append(i)
    arr_imgs = np.array(l_imgs) 
    return arr_imgs, l_idx_zeros

In [ ]:
def get_sequence(l_idx, clockwise=True):
    """Returns a reconstruction sequence as a list of integer numbers.
    Each int represents the index of a picture in a ordered list
    (l_idx).
    
    The way is spiral-shaped and its orientation is determinated by
    clockwise (bool).
    
    For squared arrays with odd number of rows, the spiral's head is
    in the center; instead, for an even number, the head is in the
    (center, center - 1) coordinate ((column, row))."""
    # Complete l_idx (list) with ones to convert it into squared
    # array.
    n = int(math.ceil(math.sqrt(len(l_idx))))
    arr = np.ones(n ** 2, dtype=int)
    arr *= -1
    for i in range(len(l_idx)):
        arr[i] = l_idx[i]
    # Split arr (numpy.ndarray) to give it n-by-n new order.
    arr = np.split(arr, n)
    arr = np.array(arr)
    # Determinate the (x, y) coordinate ((colum, row)) of the head the
    # spiral.
    if n % 2 != 0:
        col = (n - 1) // 2
        row = col
    else:
        col = (n // 2) - 1
        row = col
    addends = [(-1, -1), (1, 1)]
    if clockwise:
        i = 0  # Index in addends ('list').
    else:
        i = 1
    c = 1
    l_seq = list()
    l_seq.append(arr[row][col])  # First, add the head of the spiral to
                                 # the list.
    j = 1
    steps = 1
    while True:
        i %= 2
        for k in range(j):
            col += addends[i][0]
            el = arr[row][col]
            if el != -1:  # It does not add the -1 index.
                l_seq.append(el)
            steps += 1
            if steps >= arr.size:
                break
        if steps >= arr.size:
            break
        for k in range(j):
            row += addends[i][1]
            el = arr[row][col]
            if el != -1:
                l_seq.append(arr[row][col])
            steps += 1
            if steps >= arr.size:
                break
        if steps >= arr.size:
            break
        i += 1
        j += 1
    return l_seq

In [ ]:
def get_photo(leds, **kwargs):
    """Generates the high-quality and Hi-Res object by applying of
    recovery process of Fourier Ptychography algorithm and returns it
    as a numpy.ndarray.
    
    Args:
        leds: A number of LEDs (int) per row or column of a squared
            LED illuminator or a list of positive integer numbers that
            represent the number of LEDs per ring from the center to
            the edge in a LED ring illuminator.
        **kwargs: Keyword arguments. See inline comments.
    """
    inpath = kwargs.pop('inpath',
                        os.path.join(os.path.expanduser('~'),
                                     's3-out',
                                     'microscope',
                                     'lores-set-0001'))
    outdirname = kwargs.pop('outdirname', os.path.join(inpath,
                                                       'hq-fp-0001'))
    outpath = kwargs.pop('outpath',
                         fwd.find_out_dir(dirname=outdirname,
                                          parentdir='microscope'))
    inprefix = kwargs.pop('prefix', 'lores-img_')
    inext = kwargs.pop('inext', '.tif')  # Extension of the input
                                         # files.
    # We recommend that the output file have the same extension
    # than the input files.
    outext = kwargs.pop('outext', inext)
    # leds2use (int) is the ORDER OF A SQUARED SUBSET of Lo-Res image
    # set. This subset is aligned to the center of the main set.
    leds2use = kwargs.pop('leds2use', None)
    d = kwargs.pop('d', 5)  # Distance in mm neighboring LEDs.
    h = kwargs.pop('h', 90)  # Distance in mm between the LED grid and
                             # the sample.
    # Does the first image have 1 at the end of its filename?
    namesfrom1 = kwargs.pop('namesfrom1', True)
    ##
    # Determination of default valor for subsel (list).
    l_subsel_idxs = list()
    if isinstance(leds, int):
        # Default subset for LED grid illuminator.
        if leds2use is None or leds2use > leds:
            leds2use = leds
        l_subsel_idxs = select_centered_subset(leds ** 2,
                                               leds2use ** 2,
                                               firstis1=namesfrom1)
        l_xy_leds = fwd.gen_xy_led_grid(leds2use, d)
    elif all(isinstance(led, int) and led > 0 for led in leds):
        # Default subset for LED ring illuminator.
        if leds2use in leds:
            # Only select the photos associated to the LEDs of the
            # ring. For example, for leds = [12, 16, 36] and
            # leds2use = 16, l_subsel_idxs (list) will be equal to
            # list(range(12, 29)) or list(range(13, 30)) if namesfrom1
            # is True.
            head_l = sum(leds[:leds.index(leds2use)])
            tail_l = head_l + leds2use + 1
            if namesfrom1:
                head_l += 1
                tail_l += 1
            l_subsel_idxs = list(range(head_l, tail_l)) 
        else:
            if namesfrom1:
                l_subsel_idxs = list(range(1, sum(leds) + 1))
            else:
                l_subsel_idxs = list(range(sum(leds)))
        l_xy_leds = fwd.gen_xy_led_ring(leds, d)
    subsel = kwargs.pop('subsel', l_subsel_idxs)
    ##
    # Parameters of the coherent imaging system.
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    ##
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    hqname = kwargs.pop('hqname', 'hq-img')  # Filename of the output.
    # Does it the output image in the notebook of the calling?
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_photo.__name__,
                  list(kwargs.keys())[-1]))
    wvs = fwd.generate_wave_vectors(l_xy_leds, h)
    lores_imgs_seq, idx_zeros = get_set_from_folder(subsel,
                                    path=inpath,
                                    prefix=inprefix,
                                    extension=inext)
    # Search for the height and the width in pixels of a Lo-Res
    # picture.
    hpx = 0
    wpx = 0
    for i in range(len(lores_imgs_seq)):
        if i not in idx_zeros:
            hpx, wpx = lores_imgs_seq[i].shape
            break
    # np.zeros((0, 0)) arrays in lores_imgs_seq ('numpy.ndarray') are
    # replaced by other arrays from the same kind and wpx-by-hpx
    # shape.
    for i in idx_zeros:
        lores_imgs_seq[i] = np.zeros((hpx, wpx))
    hpx *= 4  # Height of the output Hi-Res image.
    wpx *= 4  # Width of the output Hi-Res image.
    p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx, na,
                                              wpx, hpx)
    seq = get_sequence(subsel)
    obj_recover = np.ones((hpx, wpx), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(len(seq)):
            k = seq[j]
            k -= 1
            kxc = fwd.round_half_up((wpx + 1) / 2.0 + kx[k] / dkx)
            kyc = fwd.round_half_up((hpx + 1) / 2.0 + ky[k] / dky)
            kyl = fwd.round_half_up(kyc - (p - 1) / 2.0)
            kyh = fwd.round_half_up(kyc + (p - 1) / 2.0)
            kxl = fwd.round_half_up(kxc - (q - 1) / 2.0)
            kxh = fwd.round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = (((p / hpx) ** 2)
                        * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh])
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = (((hpx / p) ** 2) * lores_imgs_seq[k]
                         * np.exp(1j * np.angle(img_lores)))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = ((1 - cft)
                                                        * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft)
    filename = hqname + outext
    img_path = os.path.join(outpath, filename)
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.toimage(obj_recover, cmin=0, cmax=255).save(img_path)
    print('Saved high-quality image:', img_path)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover

In [ ]:
def simulate_fp_proj(proj_path, nleds, **kwargs):
    """Carries out all steps of Fouerier Ptychography algorithm for an
    image set organized as SuperScanner project, i.e., each view is
    stored single within an unique folder and returns the route of the
    multiple reconstructions.
    
    Args:
        proj_path: A route (str) of a SuperScanner project.
        nleds: A number of LEDs (int) per row or column of a squared
            LED illuminator or a list of positive integer numbers that
            represent the number of LEDs per ring from the center to
            the edge in a LED ring illuminator.
        **kwargs: Keyword arguments. See inline comments.
    """
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'superscanner-software-s3'))
    dirsprename = kwargs.pop('dirprenames', 'view_')  # Valid prefix
                                                      # for folders.
    imgsprename = kwargs.pop('imgsprenames', 'view_')  # Valid prefix
                                                       # for images.
    outdirname = kwargs.pop('outdirname',
                            os.path.join(proj_path, 'fp-views-0001'))
    outpath = kwargs.pop('outpath',
                         fwd.find_out_dir(dirname=outdirname,
                             parentdir=os.path.basename(proj_path)))
    d = kwargs.pop('d', 5)  # Distance in mm between neighboring LEDs.
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(simulate_fp_proj.__name__,
                  list(kwargs.keys())[-1]))
    if isinstance(nleds, int):
        illuminatortype = 'GRID'
        n_photos = nleds ** 2
    elif all(isinstance(led, int) and led > 0 for led in nleds):
        illuminatortype = 'RING'
        n_photos = sum(nleds)
    l_ls = !ls $proj_path
    l_ss_dirs = list()  # List with the path folders that can contain
                        # an image with valid name to be used in the
                        # forward imaging model process.
    l_ss_imgs = list()  # List with the path folders THAT CONTAIN
                        # images.
    for name in l_ls:
        path = os.path.join(proj_path, name)
        if os.path.isdir(path):
            if dirsprename in name:
                l_ss_dirs.append(path)
            else:
                print(path, 'does not seem a directory with a valid',
                      'name for a SuperScanner project.')
        else:
            print(name, 'is not a directory.')
    for p in l_ss_dirs:
        l_ls = !ls $p
        for name in l_ls:
            # Forward imaging model process. See fwdimaging.ipynb
            # notebook for more details.
            path = os.path.join(p, name)
            if os.path.isfile(path):
                if imgsprename in name:
                    l_ss_imgs.append(p)
                    fwd.simulate_set(leds=nleds, amplitude=path,
                                     outpath=p)
                else:
                    print(name, 'does not seem an image with a valid',
                          'name as to be part of high-quality image',
                          'reconstruction process.')
            else:
                print(name, 'is not a file.')
    suffix1 = fwd.num_str_zeros(0, len(str(n_photos)),
                  firstis1=True)  # Suffix in the filename of the
                                  # first image file.
    out_path = outpath
    for i in range(len(l_ss_imgs)):
        suffix = fwd.num_str_zeros(int(i), len(suffix1),
                                   firstis1=True)
        img_hq_name = imgsprename + suffix
        # Recovery process. See phaseretrieval.ipynb for more details.
        get_photo(leds=nleds, inpath=l_ss_imgs[i], show=False,
                  outext='.jpg', outpath=out_path, hqname=img_hq_name)
    return out_path